### Attempt 1: Beautiful Soup

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
wsj_login_page = 'https://sso.accounts.dowjones.com/login-page?client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&redirect_uri=https%3A%2F%2Fwww.wsj.com%2Fclient%2Fauth&response_type=code&scope=openid%20idp_id%20roles%20tags%20email%20given_name%20family_name%20uuid%20djid%20djUsername%20djStatus%20trackid%20prts%20updated_at%20created_at%20offline_access&ui_locales=en-us-x-wsj-223-2&nonce=1b34552c-ac89-4605-81d9-5b552a7452a0&state=nmfLpXnMOWJLlEjK.ifKgeElwMAfxoWUUCsEZVu2q-WgzWtwQKEy_EyvQbTs&resource=https%253A%252F%252Fwww.wsj.com%252F&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#!/signin'

In [5]:
def login(username, password):
    login_url = wsj_login_page
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
    }

    # Create a session object to persist the login session
    with requests.Session() as session:
        # Send a POST request with login credentials
        login_data = {
            "username": username,
            "password": password
        }
        response = session.post(login_url, headers=headers, data=login_data)

        # Check if login was successful (you might need to customize this condition)
        if response.status_code == 200:
            return session
        else:
            print("Login failed:", response.status_code)
            return None

In [6]:
def scrape_headlines(session, technology_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
    }

    # Use the session object to access the technology section page
    response = session.get(technology_url, headers=headers)
    if response.status_code == 200:
        # Scrape the headlines from the technology section page using BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")
        headlines = [headline.text.strip() for headline in soup.find_all("a", class_="WSJTheme--headline--unZqjb45")]
        return headlines
    else:
        print("Failed to access technology section page:", response.status_code)
        return []

In [7]:
username = 'mallika101@hotmail.com'
password = 'abc123'
session = login(username, password)

Login failed: 404


In [ ]:
if session:
    technology_url = "https://www.wsj.com/news/technology"  # URL of the technology section
    headlines = scrape_headlines(session, technology_url)
    print(headlines)

### Attempt 2: Selenium

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [11]:
# Start a WebDriver (you need to have chromedriver installed in your system and its path added to the environment variables)
driver = webdriver.Edge()

# URL of the webpage to scrape
url = "https://www.wsj.com/news/archive/2024/03/21?page=1"

# Open the webpage
driver.get(url)

# Parse the HTML content
soup = BeautifulSoup(driver.page_source, "html.parser")

# Initialize lists to store data
topics = []
headlines = []
published_times = []

# Find all articles on the page
articles = soup.find_all("article")

# Extract data from each article
for article in articles:
    # Extract topic
    topic = article.find("div", class_="WSJTheme--articleType--34Gt-vdG").text.strip()
    topics.append(topic)
    
    # Extract headline
    headline = article.find("span", class_="WSJTheme--headlineText--He1ANr9C").text.strip()
    headlines.append(headline)
    
    # Extract published time
    published_time = article.find("p", class_="WSJTheme--timestamp--22sfkNDv").text.strip()
    published_times.append(published_time)

# Create a Pandas DataFrame
data = {
    "Topic": topics,
    "Headline": headlines,
    "Published Time": published_times
}

df = pd.DataFrame(data)

# Close the WebDriver
driver.quit()

In [3]:
len(articles)

50

In [4]:
df.head(10)

,Topic,Headline,Published Time
0,Economy,Japan’s Consumer Inflation Picked Up in February,9:26 PM ET
1,Healthcare,How to Fix Your Marriage in 16 Hours,9:00 PM ET
2,Travel,United and American Had the Most Baggage Handl...,9:00 PM ET
3,Election 2024,Trump Is in a Race Against Time to Protect His...,9:00 PM ET
4,If These Résumés Could Talk,The Cats Who Helped Make a Big Hire,9:00 PM ET
5,Theater Review,‘Water for Elephants’ Review: A Sanitized Circ...,9:00 PM ET
6,Economy,Australian Households Riding Out Pain of Soari...,8:42 PM ET
7,Business,Samsonite Scouts Second Listing Venue on Risin...,8:36 PM ET
8,Economy,U.K. Consumers Prove Cautious Amid Economic Cl...,8:12 PM ET
9,Markets,Robinhood Co-Founder to Step Down as Creative ...,8:04 PM ET
